### 16th April 2021

#### Logistic Regression
Used to predict probability of a class and clasify, given the values of different independent predictor variables.

Dependent variable variable (Y) is binary, i.e. having 2 possible values of 0 or 1.

Unlike Linear Regression where Y is a continuous variable, Logistic Regression needs to have predicted Y to lie between 0 and 1. 

In [1]:
using Pkg
Pkg.add("StatsBase")
Pkg.add("ClassImbalance")
Pkg.add("ROCAnalysis")
using DataFrames
using CSV
using Plots
using GLM
using StatsBase
using Lathe
using MLBase
using ClassImbalance
using ROCAnalysis

   Updating registry at `C:\Users\HP-PC\.julia\registries\General`
  Resolving package versions...
Updating `C:\Users\HP-PC\.julia\environments\v1.5\Project.toml`
  [2913bbd2] + StatsBase v0.33.5
No Changes to `C:\Users\HP-PC\.julia\environments\v1.5\Manifest.toml`
  Resolving package versions...
  Installed UnPack ───────────────── v1.0.2
  Installed ClassImbalance ───────── v0.8.7
  Installed Lathe ────────────────── v0.0.9
  Installed KernelDensity ────────── v0.5.1
  Installed ArrayInterface ───────── v2.14.17
  Installed NLSolversBase ────────── v7.5.0
  Installed ForwardDiff ──────────── v0.10.18
  Installed SpecialFunctions ─────── v0.10.3
  Installed DataFrames ───────────── v0.20.2
  Installed CommonSubexpressions ─── v0.3.0
  Installed Tables ───────────────── v1.4.2
  Installed DataStructures ───────── v0.17.20
  Installed StatsBase ────────────── v0.32.2
  Installed FillArrays ───────────── v0.8.14
  Installed DiffEqDiffTools ──────── v1.7.0
  Installed LineSearches ───────

In [2]:
ENV["COLUMNS"] = 1000   # Enable printing of 1000 columns

1000

In [3]:
# Read the file using CSV.File and convert it to DataFrame
df = DataFrame(CSV.File("Churn_Modelling.csv"))
first(df, 5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
,Int64,Int64,String,Int64,String,String,Int64,Int64,Float64,Int64,Int64,Int64,Float64,Int64
1,1,15634602,Hargrave,619,France,Female,42,2,0.0,1,1,1,1.01349e5,1
2,2,15647311,Hill,608,Spain,Female,41,1,83807.9,1,0,1,1.12543e5,0
3,3,15619304,Onio,502,France,Female,42,8,1.59661e5,3,1,0,1.13932e5,1
4,4,15701354,Boni,699,France,Female,39,1,0.0,2,0,0,93826.6,0
5,5,15737888,Mitchell,850,Spain,Female,43,2,1.25511e5,1,1,1,79084.1,0


In [4]:
println(size(df))
describe(df)

(10000, 14)


,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Union…,Nothing,DataType
1,RowNumber,5000.5,1,5000.5,10000,,,Int64
2,CustomerId,1.56909e7,15565701,1.56907e7,15815690,,,Int64
3,Surname,,Abazu,,Zuyeva,2932,,String
4,CreditScore,650.529,350,652.0,850,,,Int64
5,Geography,,France,,Spain,3,,String
6,Gender,,Female,,Male,2,,String
7,Age,38.9218,18,37.0,92,,,Int64
8,Tenure,5.0128,0,5.0,10,,,Int64
9,Balance,76485.9,0.0,97198.5,2.50898e5,,,Float64


In [5]:
names(df)

14-element Array{Symbol,1}:
 :RowNumber
 :CustomerId
 :Surname
 :CreditScore
 :Geography
 :Gender
 :Age
 :Tenure
 :Balance
 :NumOfProducts
 :HasCrCard
 :IsActiveMember
 :EstimatedSalary
 :Exited

In [6]:
countmap(df.Exited)   # Count the classes (Counts no. of unique entries in dataframes's columns)

Dict{Int64,Int64} with 2 entries:
  0 => 7963
  1 => 2037

#### One Hot Encoding
Converting categorical variables into a form that can be fed into ML algorithms

In [7]:
Lathe.preprocess.OneHotEncode(df, :Geography)
Lathe.preprocess.OneHotEncode(df, :Gender)
select!(df, Not([:RowNumber, :CustomerId, :Surname, :Geography, :Gender, :Male]))
first(df, 5)

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,France,Spain,Germany,Female
,Int64,Int64,Int64,Float64,Int64,Int64,Int64,Float64,Int64,Bool,Bool,Bool,Bool
1,619,42,2,0.0,1,1,1,1.01349e5,1,1,0,0,1
2,608,41,1,83807.9,1,0,1,1.12543e5,0,0,1,0,1
3,502,42,8,1.59661e5,3,1,0,1.13932e5,1,1,0,0,1
4,699,39,1,0.0,2,0,0,93826.6,0,1,0,0,1
5,850,43,2,1.25511e5,1,1,1,79084.1,0,0,1,0,1


In [8]:
using Lathe.preprocess: TrainTestSplit
train, test = TrainTestSplit(df, .75)

(7489×13 DataFrame
│ Row  │ CreditScore │ Age   │ Tenure │ Balance   │ NumOfProducts │ HasCrCard │ IsActiveMember │ EstimatedSalary │ Exited │ France │ Spain │ Germany │ Female │
│      │ Int64       │ Int64 │ Int64  │ Float64   │ Int64         │ Int64     │ Int64          │ Float64         │ Int64  │ Bool   │ Bool  │ Bool    │ Bool   │
├──────┼─────────────┼───────┼────────┼───────────┼───────────────┼───────────┼────────────────┼─────────────────┼────────┼────────┼───────┼─────────┼────────┤
│ 1    │ 699         │ 39    │ 1      │ 0.0       │ 2             │ 0         │ 0              │ 93826.6         │ 0      │ 1      │ 0     │ 0       │ 1      │
│ 2    │ 850         │ 43    │ 2      │ 1.25511e5 │ 1             │ 1         │ 1              │ 79084.1         │ 0      │ 0      │ 1     │ 0       │ 1      │
│ 3    │ 822         │ 50    │ 7      │ 0.0       │ 2             │ 1         │ 1              │ 10062.8         │ 0      │ 1      │ 0     │ 0       │ 0      │
│ 4    │ 376         

#### Model Building

In [9]:
fm = @formula(Exited ~ CreditScore + Age + Tenure + Balance + NumOfProducts + HasCrCard + IsActiveMember + EstimatedSalary + Female + France + Spain)
logit = glm(fm, train, Binomial(), ProbitLink())

StatsModels.TableRegressionModel{GeneralizedLinearModel{GLM.GlmResp{Array{Float64,1},Binomial{Float64},ProbitLink},GLM.DensePredChol{Float64,LinearAlgebra.Cholesky{Float64,Array{Float64,2}}}},Array{Float64,2}}

Exited ~ 1 + CreditScore + Age + Tenure + Balance + NumOfProducts + HasCrCard + IsActiveMember + EstimatedSalary + Female + France + Spain

Coefficients:
───────────────────────────────────────────────────────────────────────────────────────
                        Coef.   Std. Error       z  Pr(>|z|)     Lower 95%    Upper 95%
───────────────────────────────────────────────────────────────────────────────────────
(Intercept)      -1.83988      0.166996     -11.02    <1e-27  -2.16719      -1.51258
CreditScore      -0.000412622  0.000184325   -2.24    0.0252  -0.000773891  -5.13526e-5
Age               0.0418184    0.00167013    25.04    <1e-99   0.038545      0.0450918
Tenure           -0.00641933   0.00612882    -1.05    0.2949  -0.0184316     0.00559294
Balance           1.480

#### Model Prediction and Evaluation

In [10]:
prediction = predict(logit, test)

2511-element Array{Union{Missing, Float64},1}:
 0.12035289828535806
 0.15191515321236995
 0.3403053484472781
 0.25968999002026155
 0.05881541782974794
 0.09601265082773891
 0.6777664281814846
 0.14069230900818075
 0.05478347992548441
 0.16948770711647654
 0.5077156626503344
 0.09841481375954636
 0.3216166395709308
 ⋮
 0.2094417049886561
 0.26780973018664717
 0.2176407364288609
 0.04336271366397042
 0.3460823939048726
 0.11289882720715783
 0.06885277272804108
 0.22342699643356062
 0.5023537279235765
 0.13659262546726927
 0.06729457585517604
 0.35005319091774234